In [129]:
# Prepare environment and load data
import pandas as pd
import nltk
import feather
import re, math
from collections import Counter

event_dict = feather.read_dataframe('../parsed_data/parsed_dict.feather')

In [130]:
## Prepare event dataframe

# re.sub(r'(^|\s)x(\w+,)', r'', x)

event_dict.drop(['NA'], inplace=True, axis=1)
event_dict = event_dict.transpose()
event_dict.columns = event_dict.loc['event']
event_dict = event_dict.reindex(event_dict.index.drop(['event']))
event_dict = event_dict.applymap(lambda z: re.sub(r'(^|\s)x(\w+,)', r'', z))
event_dict.head()

event,benghazi,iran_deal,hillary,doctrine,arab_spring,benghazi_committe
dictionary,"benghazi, attack, took, place, even, septemb, ...","joint, comprehens, plan, action, jcpoa, persia...","hillari, dian, rodham, clinton, da, born, octo...","hillari, doctrin, term, us, describ, agenda, f...","arab, spring, arab, ar, rab, al, revolutionari...","unit, state, hous, select, committe, event, su..."


In [157]:
event_dict1 = event_dict.T
event_dict2 = event_dict

# Helper function to convert text to vector
WORD = re.compile(r'\w+')

def text_to_vector(text):
     words = WORD.findall(text)
     return Counter(words)

# Helper function to compute cosine similarity
def cosine_sim(vect1, vect2):
    a = set(vect1)
    b = set(vect2)
    top = len(a & b)
    bottom = len(a | b)
    if not bottom:
        sim = 0
    else:
        sim = top / float(bottom)

    return sim

In [171]:
# Create new dataframe to contain cosine similarities
# cosim_df = pd.DataFrame.from_dict(dict([for l in list_events ]))
cosim_df = pd.DataFrame()

beng = text_to_vector(event_dict.benghazi[0])
iran = text_to_vector(event_dict.iran_deal[0])
hill = text_to_vector(event_dict.hillary[0])
doct = text_to_vector(event_dict.doctrine[0])
spring = text_to_vector(event_dict.arab_spring[0])
comm = text_to_vector(event_dict.benghazi_committe[0])
# print cosine_sim(beng, hill)

benghazi = []
iran_deal = []
hillary = []
doctrine = []
arab_spring = []
benghazi_committe = []

event_dict1 = event_dict.applymap(text_to_vector)

for index, vocabulary in event_dict1.iterrows():
    for vocab in vocabulary:
        benghazi.append(round(cosine_sim(beng, vocab), 3))
        iran_deal.append(round(cosine_sim(iran, vocab), 3))
        hillary.append(round(cosine_sim(hill, vocab), 3))
        doctrine.append(round(cosine_sim(doct, vocab), 3))
        arab_spring.append(round(cosine_sim(spring, vocab), 3))
        benghazi_committe.append(round(cosine_sim(comm, vocab), 3))
        
cosim_df['benghazi'] = benghazi
cosim_df['iran_deal'] = iran_deal
cosim_df['hillary'] = hillary
cosim_df['doctrine'] = doctrine
cosim_df['arab_spring'] = arab_spring
cosim_df['benghazi_committe'] = benghazi_committe

In [172]:
cosim_df1 = cosim_df
cosim_df1.index = list(cosim_df.columns.values)
cosim_df1

,benghazi,iran_deal,hillary,doctrine,arab_spring,benghazi_committe
benghazi,1.000,0.264,0.290,0.147,0.272,0.261
iran_deal,0.264,1.000,0.311,0.126,0.226,0.212
hillary,0.290,0.311,1.000,0.154,0.248,0.235
doctrine,0.147,0.126,0.154,1.000,0.152,0.172
arab_spring,0.272,0.226,0.248,0.152,1.000,0.205
benghazi_committe,0.261,0.212,0.235,0.172,0.205,1.000


In [159]:
# event_dict.applymap(lambda x: filter(lambda z:z[0]!='x', x.split()))

In [109]:
pd.__version__

u'0.18.1'